In [ ]:
# Solution Ex. 46 SparkSQL - V2

In [ ]:
import sys

In [ ]:
inputPath = "/data/students/bigdata-01QYD/ex_data/Ex46/data/readings.txt"
outputPath = "res_out_Ex46SparkSQLV2/"

In [ ]:
# Read the content of readings
# Create a DataFrame from persons.csv
readingsDF = spark.read.load(inputPath,\
                             format="csv",\
                             header=False,\
                             inferSchema=True)\
.withColumnRenamed("_c0","timestamp")\
.withColumnRenamed("_c1","temperature")

In [ ]:
#debug
readingsDF.show()
#readingsDF.printSchema()

In [ ]:
# Create the "elements of the windows"
# The first elements are the content of readingsDF
# The second elements are the content of readingDF with timestamp = timestamp - 60
# Keep a copy of the original timestamp
readingsDFElements2 = readingsDF.selectExpr("timestamp-60 as timestamp2", \
                                            "timestamp as origtimestamp2",\
                                            "temperature as temperatureElem2")

# The third elements are the content of readingDF with timestamp = timestamp - 120
# Keep a copy of the original timestamp
readingsDFElements3 = readingsDF.selectExpr("timestamp-120 as timestamp3", \
                                            "timestamp as origtimestamp3",\
                                            "temperature as temperatureElem3")


In [ ]:
#debug
readingsDF.show()
readingsDFElements2.show()
readingsDFElements3.show()

In [ ]:
# Join the three tables to define the windows (length 3)
# Windows with less than three elements are automatically discarded by the join 
# operation
windowsDF = readingsDF.join(readingsDFElements2, readingsDF.timestamp==readingsDFElements2.timestamp2)\
            .join(readingsDFElements3, readingsDFElements2.timestamp2==readingsDFElements3.timestamp3)

In [ ]:
#debug
windowsDF.show()

In [ ]:
#debug
windowsDF.select("timestamp", "temperature",\
                 "origtimestamp2", "temperatureElem2",\
                 "origtimestamp3", "temperatureElem3").show()

In [ ]:
# Check increasing trend 
windowsIncreasingDF = windowsDF\
.filter("temperature<temperatureElem2 AND temperatureElem2<temperatureElem3")

In [ ]:
# Debug 
windowsIncreasingDF.show()

In [ ]:
# Select only original timestamps and temperatures of the three elements of each window
result = windowsIncreasingDF.select("timestamp", "temperature",\
                 "origtimestamp2", "temperatureElem2",\
                 "origtimestamp3", "temperatureElem3")

In [ ]:
#Debug 
result.show()

In [ ]:
result.write.csv(outputPath, header=False)

In [ ]:
#!hdfs dfs -cat res_out_Ex46SparkSQLV2/part*